In [3]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

import transcripty as tpy

%matplotlib inline

# How risky is college investment?

This project is meant to highlight the importance of packaging your code nicely so that
others can build upon your work.

In my own research, I benefited that the authors of this paper, Lutz Hendricks and Oksana
Leukhina, provided their code and data in a relatively usable format. We hope that after
the next few days of instruction, you will be prepared to do an even better job at
packaging your research in an easily replicable and repeatable format.

We have written two versions of a Python package that contains the code needed to
replicate elements Section 2 in their paper. You will be asked to explore some of the
implications of their model using this code.


## Simple model of college credit accumulation

In this paper, the authors are interested in "quantifying the the relative importance of heterogeneity and uncertainty for college dropout decisions." One of their innovations beyond what previous work has done is they have a more explicit model of college enrollment, credit accumulation, and dropout decisions.

Prior to presenting their full model, the authors use a simple model that only deals with credit accumulation to motivate some of the modeling decisions they make in later sections of the paper. We will focus only on this simple model.

### Individual's problem

We describe a single individual's problem.

The individual begins as a college freshman.

They begin their college career at $t=1$ with 0 college credits, $n_0 = 0$

* The student draws an ability level, $a \sim N(0, 1)$
* And a noisy signal of that ability given by $\text{GPA} = a + \varepsilon$ where $\varepsilon \sim N(0, \sigma_\varepsilon^2)$.

Each year a student attempts to complete $n_{\text{courses}} = 12$ that are each worth $n_{\text{credits}} = 3$.

The probability that a student successfully passes any given course is given by

$$p(a_i) = \gamma_{\text{min}} + \frac{1 - \gamma_{\text{min}}}{1 + \gamma_1 \exp^{-\gamma_2 a_i}}$$

A student is awarded a degree once they have accumulated 125 credits (completed 42 courses).

If a student fails to collect a degree after 6 years, they drop out of college.

### Data

The authors use data from a proprietary microdata from the High School and Beyond survey
administered by the National Center for Education Statistics (NCES) which includes
information on a student's HS GPA, college transcript information, and financial
resources to calibrate their model.



## Exploration Exercise

The code that replicates the outcomes from Hendricks Leukhina can be found on Github at https://www.github.com/cc7768/transcripty.

If you were assigned to an odd numbered group, then you should work off of the `group_odd` branch and if you were assigned to an even numbered group, then you should off of the `group_even` branch.


Once you've downloaded the repository and switched to the appropriate branch, you can install the package by entering its folder in a terminal and running `python setup.py install`. You should then be able to work from this notebook and just import the package contents like a normal package i.e., `import transcripty as tpy` (as done at top of the notebook).


### Exercise 1



### Exercise 2

### Exercise 3